# Resolução - Titanic

## Introdução

Neste notebook pretendo resolver passo a passo o problema do Titanic. A maioria das colunas são autoexplicativas; contudo, vale explicar um pouco as seguintes colunas:

* **Survived**: 0 quando a pessoa morreu e 1 quando a pessoa sobreviveu;

* **Pclass**: classe da cabine, sendo 1 para a mais cara, 2 para meio termo e 3 para a mais barata;

* **SibSp**: número de irmãos e cônjuges (marido, esposa) do passageiro que estão abordo;

* **Parch**: número de filhos e pais do passageiro que estão abordo;

* **Fare**: taxa do passageiro;

* **Embarked**: porto de embarque, sendo C = Cherbourg, Q = Queenstown, S = Southampton.

O objetivo é construir um modelo que preveja se um passageiro, através de seus dados, pode sobreviver ou não no Titanic.

## Dados Iniciais

In [1]:
from pandas import read_csv

df = read_csv('train.csv')

df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


A coluna dizendo se a pessoa sobrebviveu ou não está na segunda posição. Vamos mover para a última pois ela é a classe.

In [2]:
df2 = df.pop('Survived')

df['Survived'] = df2

df

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S,0
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S,0
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C,1


Acredito que o nome, código do ticket e a cabine não sejam relevantes no problema. Deste modo, vou retirá-las do conjunto de dados.

In [3]:
ColunasExcluidas = ['Name', 'Ticket', 'Cabin']

df.drop(columns = ColunasExcluidas,
        inplace = True)

df

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived
0,1,3,male,22.0,1,0,7.2500,S,0
1,2,1,female,38.0,1,0,71.2833,C,1
2,3,3,female,26.0,0,0,7.9250,S,1
3,4,1,female,35.0,1,0,53.1000,S,1
4,5,3,male,35.0,0,0,8.0500,S,0
...,...,...,...,...,...,...,...,...,...
886,887,2,male,27.0,0,0,13.0000,S,0
887,888,1,female,19.0,0,0,30.0000,S,1
888,889,3,female,NaN,1,2,23.4500,S,0
889,890,1,male,26.0,0,0,30.0000,C,1


## Tratamento

Agora que temos os dados que de fato usaremos, e na ordem que desejamos, vamos verificar se há linhas repetidas e células vazias. Começando pelas linhas repetidas, temos

In [5]:
df[ df.duplicated(keep = False) ]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


Como podemos ver, não há nenhuma linha repetida. Deste modo, vejamos se há células vazias em cada coluna.

In [7]:
df[df['PassengerId'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [8]:
df[df['Pclass'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [9]:
df[df['Sex'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [ ]:
df[df['Age'].isnull()]

In [12]:
df[df['SibSp'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [13]:
df[df['Parch'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [14]:
df[df['Fare'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided


In [15]:
df[df['Embarked'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survided
61,62,1,female,38.0,0,0,80.0,NaN,1
829,830,1,female,62.0,0,0,80.0,NaN,1


In [21]:
df[df['Survived'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived


Assim, temos duas colunas com dados faltantes, sendo 'Age' uma coluna numérica e 'Embarked' uma coluna categórica. Para a coluna das idade, vamos calcular a média dos passageiros e substituir nas idades faltantes.

In [4]:
IdadeMedia = df['Age'].mean()
df['Age'].fillna(value = IdadeMedia, inplace = True)

df[df['Age'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived


Como a coluna 'Embarked' é categórica, vamos usar a moda para substituir as células nulas.

In [5]:
moda = df['Embarked'].mode()

df['Embarked'].fillna(value = 'S', inplace = True)

df[df['Embarked'].isnull()]

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Survived


Obs: a moda é 'S', e coloquei direto pois se coloco value=moda o método não realiza a substituição.

## Codificação de categorias

Terminado o tratamento de dados, vamos agora transformar as colunas de strings 'Sex' e 'Embarked' em números. Para isso, usaremos o ```LabelEncoder```. No caso do 'Embarked', poderia ter usado o One Hot Encoding, mas como este é meu primeiro projeto, vou utilizar o outro por ser mais simples.

Além disso, o Label encoder funciona quando pegamos apenas os valores nas células, tirando assim o nome das colunas. Então, é conveniente já aqui fazer a separação dos atributos e a classe.

In [6]:
atributos = df.iloc[:,0:8].values
classe = df.iloc[:,8].values

from sklearn.preprocessing import LabelEncoder

codificador1 = LabelEncoder()
codificador2 = LabelEncoder()

atributos[:,2] = codificador1.fit_transform(atributos[:,2])
atributos[:,7] = codificador2.fit_transform(atributos[:,7])

print(atributos)
print(classe)

[[1 3 1 ... 0 7.25 2]
 [2 1 0 ... 0 71.2833 0]
 [3 3 0 ... 0 7.925 2]
 ...
 [889 3 0 ... 2 23.45 2]
 [890 1 1 ... 0 30.0 0]
 [891 3 1 ... 0 7.75 1]]
[0 1 1 1 0 0 0 0 1 1 1 1 0 0 0 1 0 1 0 1 0 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1
 0 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 1 0 1 1 0 1 0 0 1 0 0 0 1 1 0 1 0 0 0 0 0
 1 0 0 0 1 1 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 1 0 0 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 1 0 0 1 0 1 1 1 1 0 0 1 0 0 0 0 0 1 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0
 0 0 1 0 1 0 0 0 1 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 1 0 0 0 0 0 1 1 1 0 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 1 0 1 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 0 0 1 1 0 1 0 1 1 1 1 0 0
 0 1 0 0 1 1 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0
 1 0 0 0 0 1 0 1 0 1 1 0 0

## Divisão entre treino e teste dos dados

Aqui faremos a divisão de treino e teste, mantendo 30% dos dados para teste como usual.

In [8]:
from sklearn.model_selection import train_test_split

xTreino, xTeste, yTreino, yTeste = train_test_split(atributos, classe,
                                                    test_size = 0.3,
                                                    random_state = 0)
xTreino

array([[858, 1, 1, ..., 0, 26.55, 2],
       [53, 1, 0, ..., 0, 76.7292, 0],
       [387, 3, 1, ..., 2, 46.9, 2],
       ...,
       [630, 3, 1, ..., 0, 7.7333, 1],
       [560, 3, 0, ..., 0, 17.4, 2],
       [685, 2, 1, ..., 1, 39.0, 2]], dtype=object)

## Árvore de decisão

Para o problema do Titanic, vamos usar a árvore de decisão para produzir as previsões e mais tarde calcular a taxa de acerto e a matriz de confusão.

In [9]:
from sklearn.tree import DecisionTreeClassifier

arvore = DecisionTreeClassifier()

arvore.fit(xTreino, yTreino)

previsoes = arvore.predict(xTeste)

previsoes

array([0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0])

## Precisão

Usando esse modelo para a fatia de teste, temos que

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score

matriz = confusion_matrix(previsoes, yTeste)

matriz

array([[130,  31],
       [ 38,  69]])

In [12]:
TaxaAcerto = accuracy_score(previsoes,yTeste)

TaxaAcerto

0.7425373134328358

74% definitivamente não é uma boa taxa de acerto, contudo esse é o primeiro problema no Kaggle que estou resolvendo (poderia usar redes neurais, mas pretendo implementá-lo depois de me acostumar com os outros modelos). Além disso, podemos atribuir a forma que codifiquei os variáveis categóricas ou o fato de ter excluído algumas colunas como possíveis causas de tal valor de taxa de acerto. Por fim, espera-se que eu obtenha a mesma taxa quando aplicar o mesmo modelo no arquivo ```test.csv```. 